# 凯利公式杠杆

- $f_i = { m_i \over {s_i}^2 }$
- $g_i = r + m_i - {{s_i}^2 \over 2}$

$r$ 为无风险利率，$m_i$ 为平均超额收益 mean excess return, $s_i$ 为 $m_i$ 的标准差standard deviation.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
def datafile_info():
    res = {}
    with open('./datafiles_info.txt', encoding='utf8') as fh:
        for line in fh:
            line = line.strip()
            if not line.startswith('AssetInfo'):
                continue
            p1 = line.find('{')
            p2 = line.find(',', p1+1)
            p3 = line.find(',', p2+1)
            code = line[p1+1:p2]
            name = line[p2+2:p3]
            res[code] = name
    return res

code2name = datafile_info()
print(code2name)

{'510050.SH': '上证50ETF', '510300.SH': '沪深300ETF', '510500.SH': '中证500ETF', '159915.SZ': '创业板ETF', '159949.SZ': '创业板50ETF', '588000.SH': '科创50ETF', '518880.SH': '黄金ETF', '159781.SZ': '双创50ETF', '159601.SZ': 'A50ETF'}


In [3]:
df = pd.read_excel('datafiles/159949.SZ_2016-07-22_2023-04-25.xlsx', header=0, index_col=0)
df.shape

(1643, 7)

In [4]:
df.head()

,timestamp,open,high,low,close,amount,volume
2016-07-22 01:30:00,1469151000,0.992,0.997,0.980,0.987,151182352.0,152786635
2016-07-25 01:30:00,1469410200,0.985,0.991,0.976,0.978,56574116.0,57524377
2016-07-26 01:30:00,1469496600,0.978,0.996,0.978,0.995,64502032.0,65106373
2016-07-27 01:30:00,1469583000,0.997,0.999,0.931,0.940,82250656.0,85506942
2016-07-28 01:30:00,1469669400,0.940,0.946,0.916,0.933,28062294.0,30032572


In [5]:
df.tail()

,timestamp,open,high,low,close,amount,volume
2023-04-19 01:30:00,1681867800,1.030,1.032,1.021,1.023,5.711292e+08,557253733
2023-04-20 01:30:00,1681954200,1.022,1.022,1.002,1.005,8.794248e+08,872025823
2023-04-21 01:30:00,1682040600,1.007,1.012,0.987,0.988,8.399854e+08,840351591
2023-04-24 01:30:00,1682299800,0.988,0.988,0.960,0.965,1.168651e+09,1204231468
2023-04-25 01:30:00,1682386200,0.967,0.969,0.940,0.944,1.041406e+09,1097391616


In [6]:
k_annual_risk_free_rate = 0.02 # 假设无风险利率年化为 2%
k_days_per_year = 252

In [7]:
def compute(code, name, start, end, df):
    daily_return = df.close.pct_change()
    avg_annual_return        = daily_return.mean() * k_days_per_year
    avg_annual_excess_return = avg_annual_return - k_annual_risk_free_rate
    sd_excess_return         = (daily_return - k_annual_risk_free_rate / k_days_per_year).std()
    annual_sd_excess_return  = sd_excess_return * np.sqrt(k_days_per_year)
    sharpe_ratio   = avg_annual_excess_return / annual_sd_excess_return
    kelly_leverage = avg_annual_excess_return / (annual_sd_excess_return ** 2)
    comp_growth_rate = avg_annual_return + sd_excess_return ** 2 / 2
    return (code, name, start, end,
            avg_annual_return, avg_annual_excess_return,
            sd_excess_return, annual_sd_excess_return, sharpe_ratio,
            kelly_leverage, comp_growth_rate, df.shape[0])

In [8]:
stats_overall = []
stats_yearly  = []
for file in ['./datafiles/159601.SZ_2021-11-08_2023-04-25.xlsx',
             './datafiles/159781.SZ_2021-07-05_2023-04-25.xlsx',
             './datafiles/159915.SZ_2011-12-09_2023-04-25.xlsx',
             './datafiles/159949.SZ_2016-07-22_2023-04-25.xlsx',
             './datafiles/510050.SH_2005-02-23_2023-04-25.xlsx',
             './datafiles/510300.SH_2012-05-28_2023-04-25.xlsx',
             './datafiles/510500.SH_2013-03-15_2023-04-25.xlsx',
             './datafiles/518880.SH_2013-07-29_2023-04-25.xlsx',
             './datafiles/588000.SH_2020-11-16_2023-04-25.xlsx',
            ]:
    last_slash = file.rfind('/')
    underscore = file.find('_', last_slash)
    code = file[last_slash+1:underscore]
    name = code2name[code]
    df = pd.read_excel(file, header=0, index_col=0)
    stats_overall.append(compute(
        code, name, df.index[0].strftime('%Y.%m.%d'), df.index[-1].strftime('%Y.%m.%d'), df))

    if 1:
        years = sorted(df.index.year.unique())
        for idx in range(0, len(years)):
            start = '{:4d}.01.01'.format(years[idx])
            end   = '{:4d}.12.31'.format(years[idx])
            part_df = df[start:end]
            stats_yearly.append(compute(code, name, start, end, part_df))

columns = ['code', 'name', 'start', 'end',
           'AvgAnnRet', 'AvgAnnExRet',
           'DayExRetSD', 'AnnExRetSD',
           'SharpeRatio', 'KellyLever', 'CompGrowthRate',
           'Size']
df_overall = pd.DataFrame(stats_overall, columns=columns)
df_yearly  = pd.DataFrame(stats_yearly, columns=columns)

# 整个历史周期

In [9]:
df_overall

,code,name,start,end,AvgAnnRet,AvgAnnExRet,DayExRetSD,AnnExRetSD,SharpeRatio,KellyLever,CompGrowthRate,Size
0,159601.SZ,A50ETF,2021.11.08,2023.04.25,-0.145527,-0.165527,0.012301,0.195267,-0.847697,-4.341228,-0.145451,357
1,159781.SZ,双创50ETF,2021.07.05,2023.04.25,-0.257432,-0.277432,0.015535,0.246612,-1.124975,-4.561727,-0.257311,440
2,159915.SZ,创业板ETF,2011.12.09,2023.04.25,0.147151,0.127151,0.020814,0.330411,0.384827,1.164692,0.147368,2763
3,159949.SZ,创业板50ETF,2016.07.22,2023.04.25,0.034474,0.014474,0.018105,0.287409,0.050359,0.175217,0.034638,1643
4,510050.SH,上证50ETF,2005.02.23,2023.04.25,0.115492,0.095492,0.016755,0.265984,0.359014,1.349755,0.115632,4420
5,510300.SH,沪深300ETF,2012.05.28,2023.04.25,0.082992,0.062992,0.014677,0.232996,0.270357,1.160355,0.083100,2655
6,510500.SH,中证500ETF,2013.03.15,2023.04.25,0.109150,0.089150,0.016994,0.269773,0.330464,1.224971,0.109294,2459
7,518880.SH,黄金ETF,2013.07.29,2023.04.25,0.061328,0.041328,0.008223,0.130529,0.316621,2.425680,0.061362,2372
8,588000.SH,科创50ETF,2020.11.16,2023.04.25,-0.083447,-0.103447,0.015612,0.247834,-0.417403,-1.684207,-0.083325,594


# 每一年

In [10]:
df_yearly[:50]

,code,name,start,end,AvgAnnRet,AvgAnnExRet,DayExRetSD,AnnExRetSD,SharpeRatio,KellyLever,CompGrowthRate,Size
0,159601.SZ,A50ETF,2021.01.01,2021.12.31,0.066970,0.046970,0.008402,0.133377,0.352164,2.640370,0.067006,40
1,159601.SZ,A50ETF,2022.01.01,2022.12.31,-0.195434,-0.215434,0.013674,0.217075,-0.992442,-4.571895,-0.195341,242
2,159601.SZ,A50ETF,2023.01.01,2023.12.31,-0.075126,-0.095126,0.009118,0.144745,-0.657200,-4.540406,-0.075085,75
3,159781.SZ,双创50ETF,2021.01.01,2021.12.31,-0.268097,-0.288097,0.014936,0.237108,-1.215045,-5.124443,-0.267985,123
4,159781.SZ,双创50ETF,2022.01.01,2022.12.31,-0.275468,-0.295468,0.017041,0.270511,-1.092259,-4.037767,-0.275323,242
5,159781.SZ,双创50ETF,2023.01.01,2023.12.31,-0.117880,-0.137880,0.010376,0.164720,-0.837060,-5.081730,-0.117826,75
6,159915.SZ,创业板ETF,2011.01.01,2011.12.31,-1.452793,-1.472793,0.018706,0.296953,-4.959687,-16.701934,-1.452618,16
7,159915.SZ,创业板ETF,2012.01.01,2012.12.31,0.048293,0.028293,0.018007,0.285846,0.098981,0.346273,0.048455,243
8,159915.SZ,创业板ETF,2013.01.01,2013.12.31,0.682928,0.662928,0.021355,0.339006,1.955507,5.768361,0.683156,238
9,159915.SZ,创业板ETF,2014.01.01,2014.12.31,0.126086,0.106086,0.016034,0.254535,0.416781,1.637423,0.126214,245


In [11]:
df_yearly.shape[0] > 50 and df_yearly[50:]

,code,name,start,end,AvgAnnRet,AvgAnnExRet,DayExRetSD,AnnExRetSD,SharpeRatio,KellyLever,CompGrowthRate,Size
50,510300.SH,沪深300ETF,2016.01.01,2016.12.31,-0.016939,-0.036939,0.012569,0.199528,-0.185132,-0.927850,-0.016860,244
51,510300.SH,沪深300ETF,2017.01.01,2017.12.31,0.212296,0.192296,0.006969,0.110624,1.738277,15.713342,0.212320,244
52,510300.SH,沪深300ETF,2018.01.01,2018.12.31,-0.278391,-0.298391,0.013974,0.221834,-1.345109,-6.063574,-0.278294,243
53,510300.SH,沪深300ETF,2019.01.01,2019.12.31,0.372615,0.352615,0.012511,0.198614,1.775379,8.938851,0.372693,244
54,510300.SH,沪深300ETF,2020.01.01,2020.12.31,0.280252,0.260252,0.014824,0.235331,1.105899,4.699340,0.280362,243
55,510300.SH,沪深300ETF,2021.01.01,2021.12.31,-0.034964,-0.054964,0.011638,0.184741,-0.297521,-1.610473,-0.034897,243
56,510300.SH,沪深300ETF,2022.01.01,2022.12.31,-0.211061,-0.231061,0.012684,0.201358,-1.147512,-5.698868,-0.210980,242
57,510300.SH,沪深300ETF,2023.01.01,2023.12.31,0.071568,0.051568,0.008558,0.135855,0.379580,2.794019,0.071604,75
58,510500.SH,中证500ETF,2013.01.01,2013.12.31,0.155547,0.135547,0.014222,0.225761,0.600400,2.659450,0.155648,193
59,510500.SH,中证500ETF,2014.01.01,2014.12.31,0.354357,0.334357,0.012469,0.197936,1.689221,8.534194,0.354435,245
